# Simple Bitcoin P2P getaddr client

In [1]:
# import all the necessary modules
import socket
import binascii
import socket
import struct
import codecs
import time
import random
import hashlib
from datetime import datetime
import io
from test_framework.messages import *
import dns.resolver

Get initial list of peers

In [2]:
# dns_addr = 'testnet-seed.bitcoin.jonasschnelli.ch' # Testnet
dns_addr = 'seed.bitcoin.sipa.be' # Mainnet
for rdata in dns.resolver.resolve(dns_addr, 'A') :
    print(rdata)

18.166.71.221
168.119.172.105
109.193.209.234
52.57.53.177
15.237.37.136
184.105.70.100
51.81.208.140
178.128.93.12
125.178.6.116
188.213.34.58
134.209.247.91
34.210.198.230
176.9.123.3
154.22.123.138
107.173.166.43
137.184.91.103
72.133.177.119
145.239.255.68
73.206.62.226
185.150.160.210
37.201.138.24
18.158.93.160
64.99.192.186
173.249.48.140
61.239.91.250


#### Methods slightly adjusted from https://github.com/bitcoin/bitcoin/tree/master/test/functional/test_framework

#### Settings

In [3]:
settings_dict = {
    'magic'           : 0xf9beb4d9, # Mainnet
#    'magic'           : 0x0b110907, # Testnet
    'dstAddr'         : "34.87.168.12",
    'dstPort'         : 8333,
    'max_n_bits'      : 0x1d00ffff,
    'version'         : 70015,
    'buff_size'       : 0x02000000,  # https://github.com/bitcoin/bitcoin/blob/60abd463ac2eaa8bc1d616d8c07880dc53d97211/src/serialize.h#L23
}

In [4]:
# from p2p.py

MESSAGEMAP = {
    b"addr"         : msg_addr,
    b"addrv2"       : msg_addrv2,
    b"block"        : msg_block,
    b"blocktxn"     : msg_blocktxn,
    b"cfcheckpt"    : msg_cfcheckpt,
    b"cfheaders"    : msg_cfheaders,
    b"cfilter"      : msg_cfilter,
    b"cmpctblock"   : msg_cmpctblock,
    b"feefilter"    : msg_feefilter,
    b"filteradd"    : msg_filteradd,
    b"filterclear"  : msg_filterclear,
    b"filterload"   : msg_filterload,
    b"getaddr"      : msg_getaddr,
    b"getblocks"    : msg_getblocks,
    b"getblocktxn"  : msg_getblocktxn,
    b"getdata"      : msg_getdata,
    b"getheaders"   : msg_getheaders,
    b"headers"      : msg_headers,
    b"inv"          : msg_inv,
    b"mempool"      : msg_mempool,
    b"merkleblock"  : msg_merkleblock,
    b"notfound"     : msg_notfound,
    b"ping"         : msg_ping,
    b"pong"         : msg_pong,
    b"sendaddrv2"   : msg_sendaddrv2,
    b"sendcmpct"    : msg_sendcmpct,
    b"sendheaders"  : msg_sendheaders,
    b"tx"           : msg_tx,
    b"verack"       : msg_verack,
    b"version"      : msg_version,
    b"wtxidrelay"   : msg_wtxidrelay,
}

In [5]:
class Message:
    __slots__ = ("addrFrom", "nPortFrom", "addrTo", "nPortTo", "nNonce", "relay", "nServices",
                 "nStartingHeight", "nTime", "nVersion", "strSubVer","bMagic", "bCommand","timestamp","with_payload")
    
    def __init__(self, bCommand = b'undefined', addrTo = '0.0.0.0', nPortTo = 0, bMagic= 0xf9beb4d9, nVersion= 70015, nServices= 0,with_payload=True):
        self.nNonce = random.getrandbits(64)
        self.nStartingHeight = 596306
        self.relay = False
        self.nPortFrom = 0
        self.addrFrom = ""
        self.bCommand = bCommand
        self.addrTo = addrTo
        self.nPortTo = nPortTo
        self.with_payload = with_payload
        self.bMagic = bMagic
        self.nVersion = nVersion
        self.nServices  = nServices

    def serialize(self):
        # header 
        header     = b""
        header     += struct.pack("!I",self.bMagic)                         # magic value                           
        header     += struct.pack("<12s",self.bCommand)                     # command
        
        # payload
        payload    = b""
        if self.with_payload:
            payload    += struct.pack("i", self.nVersion)
            payload    += struct.pack("Q", self.nServices)
            payload    += struct.pack("q", int(time.time()))
            payload    += struct.pack("Q",0);

            payload    +=b"\x00" * 10 + b"\xff" * 2
            payload    += socket.inet_aton(self.addrTo)
            payload    += struct.pack(">H",self.nPortTo)                    # Receiver Port
            payload    += struct.pack("Q",0)
            payload    += struct.pack(">16s",bytes(self.addrFrom,'utf-8'))  # Sender IP Address
            payload    += struct.pack(">H",self.nPortFrom)                  # Sender Port
        if self.with_payload or self.bCommand == b'pong' :
            payload    += struct.pack("<Q", self.nNonce)
        if self.with_payload:
            payload    += struct.pack("B",0)                                # Bytes in version string
            payload    += struct.pack("i", self.nStartingHeight)
            payload    += struct.pack("?", self.relay)
        
        # length and checksum
        length     = struct.pack("I", len(payload));
        checksum   = hashlib.sha256(hashlib.sha256(payload).digest()).digest()[:4];
        
        return header + length + checksum + payload

In [6]:
import asyncio
import queue

class SimpleGetAddrClient(asyncio.Protocol):
    def __init__(self,settings_dict, on_con_lost):
        self.on_con_lost = on_con_lost
        self.transport = None
        self.recvbuf = b""
        self.settings_dict = settings_dict
        self.peers = []

    def connection_made(self, transport):
        # safe transport connection
        self.transport = transport
        
        # write version message on connection
        message = Message(bCommand=b'version', addrTo=self.settings_dict['dstAddr'],nPortTo=self.settings_dict['dstPort'],bMagic=self.settings_dict['magic'])
        print(f"Sending {message.bCommand}")
        transport.write(message.serialize())

    def data_received(self, data):
        if(len(data) > 0):
            self.recvbuf += data
            self.on_data()
        
    def on_data(self):
        try:
            while True:
                if len(self.recvbuf) < 4:
                    return
                
                # Check if magic bytes match (mainnet, testnet3 or regtest)
                if self.recvbuf[:4] != self.settings_dict['magic'].to_bytes(4, 'big'):
                    raise ValueError("magic bytes mismatch: {} != {}".format(repr(self.settings_dict['magic']), repr(self.recvbuf)))
                    
                # Message with headers is 4 + 12 + 4 + 4 bytes long
                if len(self.recvbuf) < 4 + 12 + 4 + 4:
                    return
                
                # Extract message type
                msg_type = self.recvbuf[4:4+12].split(b"\x00", 1)[0]
                msg_len = struct.unpack("<i", self.recvbuf[4+12:4+12+4])[0]
                checksum = self.recvbuf[4+12+4:4+12+4+4]
                if len(self.recvbuf) < 4 + 12 + 4 + 4 + msg_len:
                    return
                
                # Remove headers from message
                msg = self.recvbuf[4+12+4+4:4+12+4+4+msg_len]
                
                # Check if payload is not corrupted
                th = hashlib.sha256(msg).digest()
                h = hashlib.sha256(th).digest()
                if checksum != h[:4]:
                    raise ValueError("got bad checksum " + repr(self.recvbuf))
                    
                # Received Message from buffer
                self.recvbuf = self.recvbuf[4+12+4+4+msg_len:]
                if msg_type not in MESSAGEMAP:
                    raise ValueError("Received unknown msgtype from %s:%d: '%s' %s" % (self.dstaddr, self.dstport, msg_type, repr(msg)))
                f = io.BytesIO(msg)
                t = MESSAGEMAP[msg_type]()
                t.deserialize(f)
                print(f"Received {msg_type}")
                self.on_message(t)
        except Exception as e:
            print(f"Exception {e}")
            raise
            
            
    def on_message(self, message):
        try:
            msgtype = message.msgtype.decode('ascii')
            getattr(self, 'on_' + msgtype)(message)
        except:
            print(f"ERROR delivering {repr(message)}")
            raise
            
            
    def on_version(self, message):
        msg = Message(bCommand=b'verack', addrTo=self.settings_dict['dstAddr'],nPortTo=self.settings_dict['dstPort'],bMagic=self.settings_dict['magic'])
        print(f"Sending {msg.bCommand}")
        self.transport.write(msg.serialize())
        
        
    def on_verack(self, message):
        msg = Message(bCommand=b'getaddr', addrTo=self.settings_dict['dstAddr'],nPortTo=self.settings_dict['dstPort'],bMagic=self.settings_dict['magic'],with_payload=False)
        print(f"Sending {msg.bCommand}")
        self.transport.write(msg.serialize())
        
        
    def on_sendheaders(self, message):
        pass
    
    def on_ping(self, message):
        msg = Message(bCommand=b'pong', addrTo=self.settings_dict['dstAddr'],nPortTo=self.settings_dict['dstPort'],bMagic=self.settings_dict['magic'],with_payload=False)
        print(f"Sending {msg.bCommand}")
        self.transport.write(msg.serialize())
    
    def on_pong(self, message):
        pass
    
    def on_addr(self, message):
        
        for addr in message.addrs:
            self.peers.append(addr)

    def on_feefilter(self,message):
        pass
    
    def on_sendcmpct(self, message):
        pass
    
    def on_inv(self, message):
        pass

    def connection_lost(self, exc):
        print('The server closed the connection')
        self.on_con_lost.set_result(True)
        
        
    def close_connection(self):
        self.transport.close()

    

async def main(client_, loop_):
    transport, protocol = await loop_.create_connection(
        lambda: client_,
                settings_dict['dstAddr'], settings_dict['dstPort'])
    try:
        await on_con_lost
    finally:
        transport.close()

        
loop = asyncio.get_event_loop()        
on_con_lost = loop.create_future()
client = SimpleGetAddrClient(settings_dict, on_con_lost)


future = asyncio.run_coroutine_threadsafe(main(client, loop), loop)

Sending b'version'
Received b'version'
Sending b'verack'
Received b'verack'
Sending b'getaddr'
Received b'sendheaders'
Received b'sendcmpct'
Received b'sendcmpct'
Received b'ping'
Sending b'pong'
Received b'feefilter'
Received b'inv'
Received b'inv'


In [17]:
# check if already 1000 peers were received
len(client.peers)

1000

In [13]:
client.close_connection()

The server closed the connection


In [14]:
ips = []
ports = []

for peer in client.peers:
    ips.append(peer.ip)
    ports.append(peer.port)

In [15]:
def check_connection(addr,port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM);
    sock.settimeout(1)
    try:
        sock.connect((addr,port));
        sock.close()
        return True
    except:
        return False

limit = 30
online = 0
counter = 0

for ip, port in zip(ips[1:], ports[1:]):
    counter += 1
    check = check_connection(ip,port)
    if check:
        online +=1
        print(f"Possible connection: {ip}:{port}")
    if counter >= limit:
        print(f"{online} out of {limit} connections seem to be online")
        break

Possible connection: 18.159.201.250:8333
Possible connection: 217.105.86.87:8333
Possible connection: 153.231.9.193:8333
3 out of 30 connections seem to be online


In [16]:
import pandas as pd
import numpy as np

df = pd.DataFrame(list(zip(ips, ports)),
               columns =['IP', 'Port'])
df = df.iloc[np.argsort(list(map(socket.inet_aton,df['IP'])))]

for i in range(df.shape[0]):
    print(f"{df.iloc[i]['IP']:>16}:{df.iloc[i]['Port']}")

         0.0.0.0:8333
         0.0.0.0:8333
         0.0.0.1:8333
         0.0.0.1:18444
         0.0.0.1:8333
         0.0.0.1:8333
         0.0.0.1:8333
         0.0.0.1:8333
         0.0.0.2:8333
         0.0.0.2:8333
         0.0.0.2:8333
         0.0.0.2:8333
         0.0.0.2:8333
         0.0.0.2:8333
         0.0.0.5:8333
         0.0.0.5:8333
        0.0.0.38:8333
        0.0.0.62:8333
        0.0.0.72:8333
        0.0.0.73:8333
       0.0.0.120:8333
       0.0.0.180:8333
        0.0.1.51:8333
        0.0.1.54:8333
       0.0.5.178:8333
      0.0.11.198:8333
      0.0.13.147:8333
      0.0.15.179:8333
        0.0.16.1:8333
        0.0.16.1:8333
      0.0.18.205:8333
       0.0.31.90:8333
      0.0.42.114:8333
        0.0.60.7:8333
      0.0.62.187:8333
      0.0.75.124:8333
      0.0.107.20:8333
      0.0.117.76:8333
     0.0.152.247:8333
      0.0.161.14:8333
      0.0.162.62:8333
      0.0.164.36:8333
      0.0.179.85:8333
      0.0.189.23:8333
      0.0.189.50:8333
       0.